# Managing conversation history with the ChatGPT model
This sample notebook demonstrates a couple of simple patterns you can use for managing the prompts and conversation history with the ChatGPT model.

In [ ]:
# if needed, install and/or upgrade to the latest version of the OpenAI Python library
!pip install --upgrade -r requirements.txt

In [4]:
# import os module & the OpenAI Python library for calling the OpenAI API
# please make sure you have installed required libraries via pip install -r requirements.txt
import os, dotenv
from openai  import AzureOpenAI

dotenv.load_dotenv()

True

In [5]:
# Setting up the deployment name
deployment_name = "gpt-4o-mini"

# The base URL for your Azure OpenAI resource. e.g. "https://<your resource name>.openai.azure.com"
# This is the value of the endpoint for your Azure OpenAI resource
azure_endpoint = os.environ['AZURE_OPENAI_ENDPOINT']

# The API key for your Azure OpenAI resource.
api_key = os.environ['AZURE_OPENAI_API_KEY']

# Currently OPENAI API have the following versions available: https://learn.microsoft.com/azure/ai-services/openai/reference
api_version = "2024-07-01-preview"

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key = os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version="2024-02-15-preview"
)

## 1.0 Create the system message for ChatGPT

In [6]:
base_system_message = "You are a helpful assistant."

system_message = f"{base_system_message.strip()}"
print(system_message)

You are a helpful assistant.


## 2.0 Define helper functions



In [7]:
import tiktoken

def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0301"):
    encoding = tiktoken.encoding_for_model(model)
    num_tokens = 0
    for message in messages:
        num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":  # if there's a name, the role is omitted
                num_tokens += -1  # role is always required and always 1 token
    num_tokens += 2  # every reply is primed with <im_start>assistant
    return num_tokens

In [31]:
# Defining a function to send the prompt to the ChatGPT model
# More info : https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/chatgpt?pivots=programming-language-chat-completions
def send_message(messages, model_name, max_response_tokens=500, temperature=0):
    response = client.chat.completions.create(
        model=model_name,
        messages=messages,
        max_tokens=max_response_tokens,
        temperature=temperature
    )
    return response.choices[0].message.content

# Defining a function to print out the conversation in a readable format
def print_conversation(messages):
    for message in messages:
        print(f"[{message['role'].upper()}]")
        print(message['content'])
        print()

## 3.0 Start the conversation

In [9]:
# This is the first user message that will be sent to the model. Feel free to update this.
user_message = "I want to write a blog post about the impact of AI on the future of work."

In [10]:
# Create the list of messages. role can be either "user" or "assistant" 
messages = [
    {"role": "system", "content": base_system_message},
    {"role": "user", "content": user_message} 
]
print_conversation(messages)

[SYSTEM]
You are a helpful assistant.

[USER]
I want to write a blog post about the impact of AI on the future of work.



In [11]:
token_count = num_tokens_from_messages(messages)
print(f"Token count: {token_count}")

Token count: 36


In [12]:
max_response_tokens = 100

response = send_message(messages, deployment_name, max_response_tokens)
messages.append({"role": "assistant", "content": response})

print_conversation(messages)

[SYSTEM]
You are a helpful assistant.

[USER]
I want to write a blog post about the impact of AI on the future of work.

[ASSISTANT]
### The Impact of AI on the Future of Work

As we navigate through the early decades of the 21st century, the rapid advancement of artificial intelligence (AI) is reshaping our workplaces and transforming the way we perform our jobs. From automating repetitive tasks to enhancing decision-making and creativity, AI's impact on the future of work is profound and multi-faceted. In this blog post, we’ll explore how AI is influencing various aspects of the workplace and what it means for employees, businesses



## 4.0 Continue the conversation

When working with the ChatGPT model, it's your responsibity to make sure you stay within the token limits of the model. The model can handle a maximum of 4096 tokens, and this includes the number of tokens in the prompt as well as the `max_tokens` you're requesting from the model. If you exceed these limits, the model will return an error.

You should also consider the trade-off between maintaining more of the conversation history and the cost/latency that you'll incur by including those tokens in the prompt. Shorter prompts are cheaper and faster. The amount of the previous conversation you include also makes a difference in how the model responds.

In this notebook, we'll show two strategies for managing the conversation history when working with the ChatGPT model.
- Option 1: Keep the conversation within a given token limit
- Option 2: Keep the conversation within a given number of turns

### Option 1: Keep the conversation within a given token limit

`overall_max_tokens` is the maximum number of tokens that you want to include in the prompt. Th maximum number this can be set to is 4096 but you can also consider reducing this number to reduce the cost and latency of the request.

In [13]:
overall_max_tokens = 1096
prompt_max_tokens = overall_max_tokens - max_response_tokens

You can continue the conversation below by editing the user_message and running the cell as many times as you would like.

In [14]:
user_message = "The target audience for the blog post should be business leaders working in the tech industry."
#user_message = "Let's talk about generative AI and keep the tone informational but also friendly."
#user_message = "Show me a few more examples"
messages.append({"role": "user", "content": user_message})

token_count = num_tokens_from_messages(messages)
print(f"Token count: {token_count}")

# remove first message while over the token limit
while token_count > prompt_max_tokens:
    messages.pop(0)
    token_count = num_tokens_from_messages(messages)

response = send_message(messages, deployment_name)

#Append assistance response 
messages.append({"role": "user", "content": response})
print_conversation(messages)

Token count: 164
[SYSTEM]
You are a helpful assistant.

[USER]
I want to write a blog post about the impact of AI on the future of work.

[ASSISTANT]
### The Impact of AI on the Future of Work

As we navigate through the early decades of the 21st century, the rapid advancement of artificial intelligence (AI) is reshaping our workplaces and transforming the way we perform our jobs. From automating repetitive tasks to enhancing decision-making and creativity, AI's impact on the future of work is profound and multi-faceted. In this blog post, we’ll explore how AI is influencing various aspects of the workplace and what it means for employees, businesses

[USER]
The target audience for the blog post should be business leaders working in the tech industry.

[USER]
### The Impact of AI on the Future of Work: A Guide for Tech Industry Leaders

As business leaders in the tech industry, you are at the forefront of one of the most significant transformations in the workplace—artificial intellige

### Option 2: Keep the conversation within a given number of turns

In [15]:
max_messages = 10

overall_max_tokens = 4096
prompt_max_tokens = overall_max_tokens - max_response_tokens

You can continue the conversation below by editing the user_message and running the cell as many times as you would like.

In [16]:
user_message = "Make the post about generative AI aimed at business leaders who have some knowledge of the technology."
messages.append({"role": "user", "content": user_message})

while len(messages) > max_messages:
    messages.pop(0)

token_count = num_tokens_from_messages(messages)

while token_count > prompt_max_tokens:
    # remove first message from messages
    messages.pop(0)
    token_count = num_tokens_from_messages(messages)

response = send_message(messages, deployment_name)

#Append assistant response
messages.append({"role": "user", "content": response})
print_conversation(messages)


[SYSTEM]
You are a helpful assistant.

[USER]
I want to write a blog post about the impact of AI on the future of work.

[ASSISTANT]
### The Impact of AI on the Future of Work

As we navigate through the early decades of the 21st century, the rapid advancement of artificial intelligence (AI) is reshaping our workplaces and transforming the way we perform our jobs. From automating repetitive tasks to enhancing decision-making and creativity, AI's impact on the future of work is profound and multi-faceted. In this blog post, we’ll explore how AI is influencing various aspects of the workplace and what it means for employees, businesses

[USER]
The target audience for the blog post should be business leaders working in the tech industry.

[USER]
### The Impact of AI on the Future of Work: A Guide for Tech Industry Leaders

As business leaders in the tech industry, you are at the forefront of one of the most significant transformations in the workplace—artificial intelligence (AI). The int

## Ejercicio 1: Modificar el Mensaje del Sistema
Enunciado:
Crea un mensaje del sistema que defina a ChatGPT como un "analista financiero experto". Imprime el mensaje del sistema resultante y utilízalo en un mensaje inicial del usuario que pregunte:
"¿Cuáles son las tendencias actuales en el mercado de acciones tecnológicas?".


In [17]:

# Mensaje del sistema
base_system_message = "You are a financial analyst expert."
system_message = f"{base_system_message.strip()}"
print(system_message)

# Crear lista de mensajes
user_message = "¿Cuáles son las tendencias actuales en el mercado de valores tecnológico?"
messages = [{"role": "user", "content": user_message}]

# Enviar mensaje y recibir respuesta
response = send_message(messages, deployment_name)

messages.append({"role": "assistant", "content": response})
print_conversation(messages)


You are a financial analyst expert.
[USER]
¿Cuáles son las tendencias actuales en el mercado de valores tecnológico?

[ASSISTANT]
Hasta octubre de 2023, las tendencias en el mercado de valores tecnológico incluyen varios factores clave:

1. **Inteligencia Artificial (IA)**: Las empresas de tecnología están invirtiendo fuertemente en IA y aprendizaje automático. La adopción de soluciones basadas en IA para la automatización, análisis de datos y personalización está impulsando el crecimiento de este sector.

2. **Ciberseguridad**: Con el aumento de las amenazas cibernéticas, la demanda por soluciones de ciberseguridad sigue en auge. Las empresas que ofrecen tecnología para proteger datos y sistemas están experimentando un crecimiento significativo.

3. **5G y Conectividad**: La implementación de redes 5G está transformando industrias. Las empresas que desarrollan infraestructura y dispositivos compatibles con 5G están capitalizando esta tendencia.

4. **Sostenibilidad y Tecnología Verde*

## Ejercicio 2: Contar Tokens en una Conversación Extendida
Enunciado:
Dado un conjunto inicial de mensajes, agrega tres mensajes adicionales:

"Explain the impact of inflation on stock prices."
"What are the top stocks to watch in 2025?"
"Summarize the latest market news in 50 words."
Calcula y muestra el número total de tokens tras añadir cada mensaje.


In [18]:

# Mensajes iniciales
messages = [
    {"role": "system", "content": "You are a helpful assistant specialized in financial topics."},
    {"role": "user", "content": "What are the current trends in the tech stock market?"}
]

# Mensajes adicionales
additional_messages = [
    "Explica el impacto de la inflación en los precios de las acciones.",
    "¿Cuáles son las principales acciones a seguir en 2025?",
    "Resume las últimas noticias del mercado en 50 palabras."
]
# Agregar mensajes y calcular tokens
for msg in additional_messages:
    messages.append({"role": "user", "content": msg})
    token_count = num_tokens_from_messages(messages)
    print(f"Token count after adding '{msg}': {token_count}")


Token count after adding 'Explica el impacto de la inflación en los precios de las acciones.': 54
Token count after adding '¿Cuáles son las principales acciones a seguir en 2025?': 74
Token count after adding 'Resume las últimas noticias del mercado en 50 palabras.': 91


## Ejercicio 3: Personalizar el Mensaje del Sistema Según el Usuario
Enunciado:
Escribe un programa que permita al usuario definir el rol de ChatGPT mediante una entrada de texto (input). Una vez definido, responde al mensaje:
"What are your key skills as an assistant?"
Usa el mensaje del sistema definido por el usuario.


In [19]:

# Entrada del usuario para definir el mensaje del sistema
user_defined_role = input("Define the role of the assistant: ")
system_message = f"You are {user_defined_role.strip()}."

# Crear lista de mensajes
messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": "¿Cuáles son tus habilidades clave como asistente?"}
]

# Enviar mensaje y recibir respuesta
response = send_message(messages, deployment_name)

messages.append({"role": "assistant", "content": response})

# Mostrar la conversación
print_conversation(messages)


[SYSTEM]
You are comentarista deportivo.

[USER]
¿Cuáles son tus habilidades clave como asistente?

[ASSISTANT]
Como asistente, mis habilidades clave incluyen:

1. **Análisis de Datos**: Puedo procesar y analizar grandes volúmenes de información para ofrecer estadísticas y tendencias relevantes en deportes.

2. **Conocimiento Deportivo**: Tengo un amplio conocimiento sobre diversas disciplinas, desde las reglas hasta la historia y los eventos más destacados.

3. **Comunicación Clara**: Puedo explicar conceptos complejos de manera sencilla, lo que es esencial para transmitir información a la audiencia.

4. **Actualización Constante**: Estoy entrenado con información más reciente hasta octubre de 2023, lo que me permite ofrecer análisis actuales y relevantes.

5. **Predicciones y Proyecciones**: Puedo utilizar datos y estadísticas para hacer predicciones sobre el rendimiento de equipos y jugadores, analizando factores como forma reciente, lesiones y emparejamientos.

6. **Comprensión de 

## Ejercicio 4: Crear un Generador de Respuestas Creativo
Enunciado:
Diseña un programa que permita configurar la creatividad del modelo (temperatura) a través de una entrada del usuario. Una vez configurado, responde a la pregunta:
"Can you suggest an out-of-the-box idea for a startup?"



In [40]:
# Entrada del usuario para definir la creatividad
temperature = float(input("Set the creativity level (0.0 - 1.0): "))
if not (0.0 <= temperature <= 1.0):
    raise ValueError("Creativity level must be between 0.0 and 1.0.")

# Crear lista de mensajes
system_message = "You are a highly creative assistant."
messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": "¿Puedes sugerir una idea innovadora para una startup?"}
]

# Enviar mensaje con creatividad ajustada
response = send_message(messages, deployment_name, max_response_tokens, temperature)

# Mostrar la respuesta
print(response)

¡Por supuesto! Aquí tienes una idea innovadora para una startup:

**Plataforma de Conexión Intergeneracional: "Sabiduría Compartida"**

**Concepto:**
Desarrolla una plataforma en línea que conecte a personas mayores con jóvenes, promoviendo el intercambio de conocimientos, experiencias y habilidades. La idea es crear un espacio donde las generaciones puedan aprender unas de otras, fomentando el entendimiento mutuo y fortaleciendo la cohesión social.

**Funcionalidades


## Ejercicio 5: Cambiar el Tono del Mensaje
Enunciado:
Configura el mensaje del sistema para que ChatGPT responda en un tono humorístico y escribe una respuesta al mensaje:
"Explain quantum computing in a funny way."


In [47]:

# Mensaje del sistema humorístico
system_message = "You are a humorous assistant."
messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": "Explica la computación cuántica de una manera divertida."}
]

# Enviar mensaje y recibir respuesta
response = send_message(messages, deployment_name)

#append assistant response
messages.append({"role": "assistant", "content": response})
print_conversation(messages)


[SYSTEM]
You are a humorous assistant.

[USER]
Explica la computación cuántica de una manera divertida.

[ASSISTANT]
¡Claro! Imagina que la computación cuántica es como una fiesta de disfraces en la que todos los asistentes son partículas subatómicas. En esta fiesta, las partículas no solo pueden estar en un lugar, sino que pueden estar en varios lugares al mismo tiempo, como si estuvieran haciendo un "moonwalk" cuántico. ¡Es como si Michael Jackson hubiera sido un electrón!

En el mundo clásico, los bits son como monedas: pueden ser cara (0) o cruz (1). Pero en la computación cuántica, los qubits son como monedas que están girando en el aire. No sabes si caerán en cara o cruz hasta que las observes. ¡Es como si tuvieras una moneda mágica que, al lanzarla, te dice que es ambas cosas a la vez hasta que la atrapes!

Además, en esta fiesta cuántica, las partículas pueden "entrelazarse", lo que significa que si tienes dos qubits entrelazados, lo que le pase a uno afecta al otro, sin import

## Ejercicio 6: Resumir Mensajes Previos
Enunciado:
Dado un historial de conversación de 5 mensajes, crea una función que resuma el contenido de los mensajes del usuario en una sola oración.



In [46]:

# Historial de mensajes
messages = [
    {"role": "system", "content": "Eres un asistente útil."},
    {"role": "user", "content": "Háblame sobre la inteligencia artificial."},
    {"role": "user", "content": "¿Cómo funciona el aprendizaje automático?"},
    {"role": "user", "content": "¿Puedes explicar las redes neuronales?"},
    {"role": "user", "content": "¿Cuáles son las aplicaciones del aprendizaje profundo?"}
]

# Resumir mensajes del usuario
def summarize_user_messages(messages):
    user_messages = [msg['content'] for msg in messages if msg['role'] == 'user']
    summary_prompt = "Resume los siguientes mensajes en una sola frase: " + " ".join(user_messages)
    summary = send_message([{"role": "system", "content": "You are a helpful assistant."},
                            {"role": "user", "content": summary_prompt}], deployment_name)
    return summary

# Obtener y mostrar el resumen
summary = summarize_user_messages(messages)
print(f"User messages summary: {summary}")


User messages summary: La inteligencia artificial se basa en el aprendizaje automático, que utiliza redes neuronales para procesar datos y tiene aplicaciones en diversas áreas como la visión por computadora, el procesamiento del lenguaje natural y la automatización.


## Ejercicio 7: Configuración de Temperatura y Tono
Enunciado:
Escribe un programa que cambie la configuración de temperatura a 0.9 para obtener respuestas más creativas y responde al mensaje:
"Suggest a unique title for a blog about AI in education."



In [45]:

# Mensajes iniciales
messages = [
    {"role": "system", "content": "You are a creative assistant."},
    {"role": "user", "content": "Sugerir un título único para un blog sobre IA en la educación."}
]
temperature = float(0.9)

# Ajustar temperatura y enviar mensaje
response = send_message(messages, deployment_name, max_response_tokens, temperature)

# Mostrar la respuesta
print(response)


"Inteligencia Artificial: Transformando el Aula del Futuro"
